In [ ]:
import cv2 import os
import numpy as np
import random
import hashlib
from skimage.metrics import structural_similarity as ssim
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

def analyze_image(image_path):
    model = VGG16(weights='imagenet', include_top=False)
    img = cv2.imread(image_path)
    if img is None:
        return 0
    img = cv2.resize(img, (224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array)
    return np.mean(features)

os.chdir("E:/")
print("Current directory:", os.getcwd())

image_folder = "images/"
if not os.path.exists(image_folder):
    print(f"Error: {image_folder} not found!")
    exit()

image_files = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))]

if not image_files:
    print("No images found in the folder.")
    exit()

best_image = max(image_files, key=analyze_image)
print(f"Best image selected: {best_image}")

img = cv2.imread(best_image)
if img is None:
    print("Error: Failed to load the selected image.")
    exit()

original_img = img.copy()

msg = input("Enter secret message: ")
password = input("Enter a passcode: ")

d = {chr(i): i for i in range(256)}

h, w, _ = img.shape
max_chars = h * w

if len(msg) > max_chars:
    print("Error: Message is too long for this image. Try a shorter message or a larger image.")
    exit()

positions = [(i % w, i // w, i % 3) for i in range(len(msg))]
random.shuffle(positions)

for i, char in enumerate(msg):
    x, y, z = positions[i]
    img[y, x, z] = d.get(char, 0) % 256

cv2.imwrite("encryptedImage.png", img)
os.system("start encryptedImage.png")

hashed_password = hashlib.sha256(password.encode()).hexdigest()

with open("key.txt", "w") as f:
    f.write(hashed_password)

with open("positions.txt", "w") as f:
    for pos in positions:
        f.write(f"{pos[0]},{pos[1]},{pos[2]}\n")

print("Encryption complete! The message has been hidden.")
